# Checking presidential corporate card (Brazil)
<pre>
Data Source https://www.gov.br/secretariageral/pt-br/acesso-a-informacao/informacoes-classificadas-e-desclassificadas/Planilha12003a2022.csv
</pre>

In [1]:
import pandas as pd
import plotly 
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter
import matplotlib.dates as mdates

In [2]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('R$', '').replace('.', '').replace(' ','').replace(',','.'))
    return(x)

In [3]:
# Read Presidential corporate card
df=pd.read_csv('Planilha12003a2022.csv.gz', sep=";", encoding="latin-1")[['DATA PGTO','VALOR']]
df = df.rename(columns={'DATA PGTO':'date','VALOR':'value'})


In [ ]:
# Convert values to numbers
df['value']=df['value'].apply(clean_currency)
df=df[~df['value'].isna()]

try:
    df['value'] = pd.to_numeric(df['value']) 
except ValueError:
    # I want to register on my log the message recived on ORIGINAL VALUE
    mask = pd.to_numeric(df['value'], errors='coerce').isna() 
    #if possible missing values
    #mask = pd.to_numeric(df['ORIGINAL_VALUE'].fillna('0'), errors='coerce').isna() 
    L = df.loc[mask, 'value'].tolist()
    #print ("Not converted values are: " + ", ".join(L))
    #Not converted values are: Wrong format
    print(f"Ignoring not converted values are: {L}")

In [ ]:
# Attempt to convert 'value' to numeric, coercing errors
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# Downcast numeric types to the smallest size (forget nonums)
df['value'] = pd.to_numeric(df['value'], downcast='float').fillna(0)   

In [ ]:
# convert date
df['date']=pd.to_datetime(df['date'],format='%d/%m/%Y')

# Presidential expenses by day

In [ ]:
data=df.set_index('date')
fig, ax = plt.subplots()
data.plot(title="Gastos Diários com cartao corporativo 2003-2022",figsize=(25, 6), fontsize=8, ax=ax)
ax.legend(["Valor em Reais"]);
ax.xaxis.set_major_locator(mdates.YearLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))

fig.savefig('diario.png')

# Expenses by year

In [ ]:
data = df.groupby(df.date.dt.year)['value'].agg(['sum'])

In [ ]:
# Convert values to millions
data['sum'] /= 1000000

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
data.plot(kind='bar', title="Gastos Anuais com cartao corporativo 2003-2022",figsize=(15, 6), fontsize=12, ax=ax)
ax.legend(["Valor em Milhões de Reais"]);
#data.plot(kind='bar', ax=ax)
plt.title('Gastos por ano')
plt.xlabel('Ano')
plt.ylabel('Valor (em milhões)')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
fig.savefig('anual.png')